In [ ]:
!ls

sample_data


# Initialize PySpark Steps

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main a

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

In [ ]:
!ls

sample_data  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz


# Read Data From Google Drive Steps

## STEP-1: Import Libraries

In [ ]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

## STEP-2: Autheticate E-Mail ID



In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## STEP-3: Get File from Drive using file-ID

In [ ]:
downloaded = drive.CreateFile({'id':'1NMX2aeVHyh3TmcbiejUFteDUpsCMSjYG'}) # replace the id with id of file you want to access
downloaded.GetContentFile('crime.csv') 

In [ ]:
downloaded1 = drive.CreateFile({'id':'1xYCedvCOFkblbTIS0pCTRDJYyAVbfzS5'}) # replace the id with id of file you want to access
downloaded1.GetContentFile('reported-crimes.csv') 

In [ ]:
!ls

adc.json   reported-crimes.csv	spark-2.3.1-bin-hadoop2.7
crime.csv  sample_data		spark-2.3.1-bin-hadoop2.7.tgz


# Read The File in PySpark

In [ ]:
!ls

adc.json   reported-crimes.csv	spark-2.3.1-bin-hadoop2.7      spark-warehouse
crime.csv  sample_data		spark-2.3.1-bin-hadoop2.7.tgz


In [ ]:
crime = spark.read.csv('crime.csv',header=True)
crime.show(5)

+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+
|       TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|   HUNDRED_BLOCK|NEIGHBOURHOOD|       X|         Y|   Latitude|   Longitude|
+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+
|Other Theft|2003|    5| 12|  16|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    5|  7|  15|    20|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 23|  16|    40|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 20|  11|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 12|  17|    45|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
+-----------+----+-----+---+----+------+----------------+-------

In [ ]:
crime.printSchema()

root
 |-- TYPE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- HOUR: string (nullable = true)
 |-- MINUTE: string (nullable = true)
 |-- HUNDRED_BLOCK: string (nullable = true)
 |-- NEIGHBOURHOOD: string (nullable = true)
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [ ]:
crime.count()

530652

In [ ]:
crime.columns

['TYPE',
 'YEAR',
 'MONTH',
 'DAY',
 'HOUR',
 'MINUTE',
 'HUNDRED_BLOCK',
 'NEIGHBOURHOOD',
 'X',
 'Y',
 'Latitude',
 'Longitude']

In [ ]:
from pyspark.sql.functions import to_timestamp,col
rc = spark.read.csv('reported-crimes.csv',header=True).withColumn('Date',to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss')).withColumn('Updated On',to_timestamp(col('Updated On'),'MM/dd/yyyy hh:mm:ss'))

In [ ]:
rc.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|         Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|    

In [ ]:
rc.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: timestamp (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
crime.select('Type','Year').show(5)

+-----------+----+
|       Type|Year|
+-----------+----+
|Other Theft|2003|
|Other Theft|2003|
|Other Theft|2003|
|Other Theft|2003|
|Other Theft|2003|
+-----------+----+
only showing top 5 rows



# Adding a new column with a constant literal value

In [ ]:
from pyspark.sql.functions import lit

rc_new = rc.withColumn('newCol',lit(1))
rc_new.show(5)

+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+------+
|      ID|Case Number|               Date|               Block|IUCR|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|         Updated On|Latitude|Longitude|Location|newCol|
+--------+-----------+-------------------+--------------------+----+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL|1153| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|       

# Remove/Drop a Column 

In [ ]:
rc_new = rc_new.drop('IUCR')
rc_new.show(5)

+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+------+
|      ID|Case Number|               Date|               Block|       Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|         Updated On|Latitude|Longitude|Location|newCol|
+--------+-----------+-------------------+--------------------+-------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+-------------------+--------+---------+--------+------+
|11034701|   JA366925|2001-01-01 11:00:00|     016XX E 86TH PL| DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|           RESIDENCE| false|   false|0412|     004|   8|            45|      11|   

# We will using Crime.csv with group by

In [ ]:
crime.show(5)

+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+
|       TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|   HUNDRED_BLOCK|NEIGHBOURHOOD|       X|         Y|   Latitude|   Longitude|
+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+
|Other Theft|2003|    5| 12|  16|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    5|  7|  15|    20|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 23|  16|    40|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 20|  11|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
|Other Theft|2003|    4| 12|  17|    45|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|
+-----------+----+-----+---+----+------+----------------+-------

In [ ]:
crime.groupBy('TYPE','YEAR').count().orderBy(['Year','count'],ascending=[False,False]).show(25)

+--------------------+----+-----+
|                TYPE|YEAR|count|
+--------------------+----+-----+
|  Theft from Vehicle|2017| 5962|
|         Other Theft|2017| 2866|
|            Mischief|2017| 2581|
|Offence Against a...|2017| 1605|
|Break and Enter R...|2017| 1304|
|Break and Enter C...|2017| 1111|
|    Theft of Bicycle|2017|  986|
|    Theft of Vehicle|2017|  755|
|Vehicle Collision...|2017|  671|
|            Homicide|2017|   11|
|Vehicle Collision...|2017|    5|
|  Theft from Vehicle|2016|12806|
|         Other Theft|2016| 5708|
|            Mischief|2016| 4599|
|Offence Against a...|2016| 3172|
|Break and Enter R...|2016| 2994|
|Break and Enter C...|2016| 2686|
|    Theft of Bicycle|2016| 2634|
|Vehicle Collision...|2016| 1699|
|    Theft of Vehicle|2016| 1474|
|Vehicle Collision...|2016|   15|
|            Homicide|2016|   11|
|  Theft from Vehicle|2015|10544|
|         Other Theft|2015| 4679|
|            Mischief|2015| 4193|
+--------------------+----+-----+
only showing t

#PySpark Built-in Functions

In [ ]:
from pyspark.sql import functions
print(dir(functions))

['AutoBatchedSerializer', 'Column', 'DataFrame', 'DataType', 'PandasUDFType', 'PickleSerializer', 'PythonEvalType', 'SparkContext', 'StringType', 'UserDefinedFunction', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_binary_mathfunctions', '_collect_list_doc', '_collect_set_doc', '_create_binary_mathfunction', '_create_function', '_create_udf', '_create_window_function', '_functions', '_functions_1_4', '_functions_1_6', '_functions_2_1', '_functions_deprecated', '_lit_doc', '_message', '_string_functions', '_test', '_to_java_column', '_to_seq', '_window_functions', '_wrap_deprecated_function', 'abs', 'acos', 'add_months', 'approxCountDistinct', 'approx_count_distinct', 'array', 'array_contains', 'asc', 'ascii', 'asin', 'atan', 'atan2', 'avg', 'base64', 'bin', 'bitwiseNOT', 'blacklist', 'broadcast', 'bround', 'cbrt', 'ceil', 'coalesce', 'col', 'collect_list', 'collect_set', 'column', 'concat', 'concat_ws', 'conv', 'c

In [ ]:
from pyspark.sql.functions import mean,sum,max

In [ ]:
crime.select(max(col('YEAR'))).show()

+---------+
|max(YEAR)|
+---------+
|     2017|
+---------+



## String Functions

In [ ]:
from pyspark.sql.functions import upper,lower,substring

crime.select(upper(col('TYPE')),lower(col('TYPE')),substring(col('TYPE'),1,5)).show(10)

+--------------------+--------------------+---------------------+
|         upper(TYPE)|         lower(TYPE)|substring(TYPE, 1, 5)|
+--------------------+--------------------+---------------------+
|         OTHER THEFT|         other theft|                Other|
|         OTHER THEFT|         other theft|                Other|
|         OTHER THEFT|         other theft|                Other|
|         OTHER THEFT|         other theft|                Other|
|         OTHER THEFT|         other theft|                Other|
|         OTHER THEFT|         other theft|                Other|
|BREAK AND ENTER R...|break and enter r...|                Break|
|            MISCHIEF|            mischief|                Misch|
|         OTHER THEFT|         other theft|                Other|
|BREAK AND ENTER R...|break and enter r...|                Break|
+--------------------+--------------------+---------------------+
only showing top 10 rows



In [ ]:
help(substring)

Help on function substring in module pyspark.sql.functions:

substring(str, pos, len)
    Substring starts at `pos` and is of length `len` when str is String type or
    returns the slice of byte array that starts at `pos` in byte and is of length `len`
    when str is Binary type.
    
    .. note:: The position is not zero based, but 1 based index.
    
    >>> df = spark.createDataFrame([('abcd',)], ['s',])
    >>> df.select(substring(df.s, 1, 2).alias('s')).collect()
    [Row(s='ab')]
    
    .. versionadded:: 1.5



In [ ]:
from pyspark.sql.functions import date_add,date_format,concat,col,lit,to_date,to_timestamp
print(help(date_add),'\n\n',help(date_format))

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]
    
    .. versionadded:: 1.5

Help on function date_format in module pyspark.sql.functions:

date_format(date, format)
    Converts a date/timestamp/string to a value of string in the format specified by the date
    format given by the second argument.
    
    A pattern could be for instance `dd.MM.yyyy` and could return a string like '18.03.1993'. All
    pattern letters of the Java class `java.text.SimpleDateFormat` can be used.
    
    .. note:: Use when ever possible specialized functions like `year`. These benefit from a
        specialized implementation.
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_format('dt', 'MM/dd

In [ ]:
help(to_date)

Help on function to_date in module pyspark.sql.functions:

to_date(col, format=None)
    Converts a :class:`Column` of :class:`pyspark.sql.types.StringType` or
    :class:`pyspark.sql.types.TimestampType` into :class:`pyspark.sql.types.DateType`
    using the optionally specified format. Specify formats according to
    `SimpleDateFormats <http://docs.oracle.com/javase/tutorial/i18n/format/simpleDateFormat.html>`_.
    By default, it follows casting rules to :class:`pyspark.sql.types.DateType` if the format
    is omitted (equivalent to ``col.cast("date")``).
    
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t).alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]
    
    >>> df = spark.createDataFrame([('1997-02-28 10:30:00',)], ['t'])
    >>> df.select(to_date(df.t, 'yyyy-MM-dd HH:mm:ss').alias('date')).collect()
    [Row(date=datetime.date(1997, 2, 28))]
    
    .. versionadded:: 2.2



In [ ]:
data = [
    ('1990-05-03 13:10:00', 29, True),
    ('1994-09-23 10:19:15', 25, False)
]
dftest = spark.createDataFrame(data, ['dob', 'age', 'is_fan'])
dftest.show()

+-------------------+---+------+
|                dob|age|is_fan|
+-------------------+---+------+
|1990-05-03 13:10:00| 29|  true|
|1994-09-23 10:19:15| 25| false|
+-------------------+---+------+



In [ ]:
dftest.printSchema()

root
 |-- dob: string (nullable = true)
 |-- age: long (nullable = true)
 |-- is_fan: boolean (nullable = true)



In [ ]:
from pyspark.sql.functions import hour,minute
dftest.select(hour(col('dob')),minute(col('dob'))).filter(hour(col('dob'))=='13').show()

+---------+-----------+
|hour(dob)|minute(dob)|
+---------+-----------+
|       13|         10|
+---------+-----------+



In [ ]:
crime_new = crime.withColumn('dayofcrime',to_date(concat(col('YEAR'),lit('-'),col('MONTH'),lit('-'),col('DAY'))))
crime_new.show(5)

+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+----------+
|       TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|   HUNDRED_BLOCK|NEIGHBOURHOOD|       X|         Y|   Latitude|   Longitude|dayofcrime|
+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+----------+
|Other Theft|2003|    5| 12|  16|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-05-12|
|Other Theft|2003|    5|  7|  15|    20|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-05-07|
|Other Theft|2003|    4| 23|  16|    40|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-04-23|
|Other Theft|2003|    4| 20|  11|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-04-20|
|Other Theft|2003|    4| 12|  17|    45|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-

In [ ]:
crime_new.printSchema()

root
 |-- TYPE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- HOUR: string (nullable = true)
 |-- MINUTE: string (nullable = true)
 |-- HUNDRED_BLOCK: string (nullable = true)
 |-- NEIGHBOURHOOD: string (nullable = true)
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- dayofcrime: date (nullable = true)



In [ ]:
crime_new.select(max(col('dayofcrime'))).show()

+---------------+
|max(dayofcrime)|
+---------------+
|     2017-07-13|
+---------------+



In [ ]:
help(date_add)

Help on function date_add in module pyspark.sql.functions:

date_add(start, days)
    Returns the date that is `days` days after `start`
    
    >>> df = spark.createDataFrame([('2015-04-08',)], ['dt'])
    >>> df.select(date_add(df.dt, 1).alias('next_date')).collect()
    [Row(next_date=datetime.date(2015, 4, 9))]
    
    .. versionadded:: 1.5



In [ ]:
crime_new.select(date_add(max(col('dayofcrime')),30)).show(5)

+-----------------------------+
|date_add(max(dayofcrime), 30)|
+-----------------------------+
|                   2017-08-12|
+-----------------------------+



## Create Dataframe

In [ ]:
data = [
    ('1990-05-03', 29, True),
    ('1994-09-23', 25, False)
]
dftest = spark.createDataFrame(data, ['dob', 'age', 'is_fan'])
dftest.show()

+----------+---+------+
|       dob|age|is_fan|
+----------+---+------+
|1990-05-03| 29|  true|
|1994-09-23| 25| false|
+----------+---+------+



In [ ]:
import pyspark.sql.types as st
data = [
    ('03/May/1990', 30, True),
    ('23/Sep/1994', 25, False)
]

user_schema = st.StructType([
    st.StructField('dob', st.StringType(), True),
    st.StructField('age', st.IntegerType(), True),
    st.StructField('is_mature', st.BooleanType(), True)
])

user_df = spark.createDataFrame(data, user_schema)
user_df.show()

+-----------+---+---------+
|        dob|age|is_mature|
+-----------+---+---------+
|03/May/1990| 30|     true|
|23/Sep/1994| 25|    false|
+-----------+---+---------+



In [ ]:
user_df.printSchema()

root
 |-- dob: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- is_mature: boolean (nullable = true)



In [ ]:
user_df.select(to_date(col('dob'),'dd/MMM/yyyy')).show()

+-----------------------------+
|to_date(`dob`, 'dd/MMM/yyyy')|
+-----------------------------+
|                   1990-05-03|
|                   1994-09-23|
+-----------------------------+



In [ ]:
crime_new.show(5)

+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+----------+
|       TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|   HUNDRED_BLOCK|NEIGHBOURHOOD|       X|         Y|   Latitude|   Longitude|dayofcrime|
+-----------+----+-----+---+----+------+----------------+-------------+--------+----------+-----------+------------+----------+
|Other Theft|2003|    5| 12|  16|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-05-12|
|Other Theft|2003|    5|  7|  15|    20|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-05-07|
|Other Theft|2003|    4| 23|  16|    40|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-04-23|
|Other Theft|2003|    4| 20|  11|    15|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-123.0837633|2003-04-20|
|Other Theft|2003|    4| 12|  17|    45|9XX TERMINAL AVE|   Strathcona|493906.5|5457452.47|49.26980201|-

In [ ]:
crime_new.filter(col('YEAR') == lit(2017)).show()

+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+-----------+------------+----------+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|       NEIGHBOURHOOD|        X|         Y|   Latitude|   Longitude|dayofcrime|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+-----------+------------+----------+
|  Theft from Vehicle|2017|    5|  3|   9|    33|     13XX ALBERNI ST|            West End|490724.22|5459449.41|49.28772481|-123.1275542|2017-05-03|
|Offence Against a...|2017|    4|  4|null|  null|OFFSET TO PROTECT...|                null|        0|         0|          0|           0|2017-04-04|
|Offence Against a...|2017|    3| 27|null|  null|OFFSET TO PROTECT...|                null|        0|         0|          0|           0|2017-03-27|
|  Theft from Vehicle|2017|    3| 26|   4|    47|        35XX ETON ST|    Hastings-Sunrise|497991.27|54594

# JOINS

In [ ]:
valuesA = [('Pirate',1),('Monkey',2),('Ninja',3),('Spaghetti',4)]
TableA = spark.createDataFrame(valuesA,['name','id'])
 
valuesB = [('Rutabaga',1),('Pirate',2),('Ninja',3),('Darth Vader',4)]
TableB = spark.createDataFrame(valuesB,['name','id'])
 
TableA.show()
TableB.show()

+---------+---+
|     name| id|
+---------+---+
|   Pirate|  1|
|   Monkey|  2|
|    Ninja|  3|
|Spaghetti|  4|
+---------+---+

+-----------+---+
|       name| id|
+-----------+---+
|   Rutabaga|  1|
|     Pirate|  2|
|      Ninja|  3|
|Darth Vader|  4|
+-----------+---+



In [ ]:
ta = TableA.alias('ta')
tb = TableB.alias('tb')

## INNER JOIN

In [ ]:
inner_join = ta.join(tb, ta.name == tb.name)
inner_join.show()

+------+---+------+---+
|  name| id|  name| id|
+------+---+------+---+
| Ninja|  3| Ninja|  3|
|Pirate|  1|Pirate|  2|
+------+---+------+---+



## LEFT JOIN

In [ ]:
left_join = ta.join(tb, ta.name == tb.name,how='left') # Could also use 'left_outer'
left_join.show()

+---------+---+------+----+
|     name| id|  name|  id|
+---------+---+------+----+
|Spaghetti|  4|  null|null|
|    Ninja|  3| Ninja|   3|
|   Pirate|  1|Pirate|   2|
|   Monkey|  2|  null|null|
+---------+---+------+----+



## LEFT JOIN WITH NULL

In [ ]:
left_join = ta.join(tb, ta.name == tb.name,how='left') # Could also use 'left_outer'
left_join.filter(col('tb.name').isNull()).show()

+---------+---+----+----+
|     name| id|name|  id|
+---------+---+----+----+
|Spaghetti|  4|null|null|
|   Monkey|  2|null|null|
+---------+---+----+----+



## RIGHT JOIN

In [ ]:
right_join = ta.join(tb, ta.name == tb.name,how='right') # Could also use 'right_outer'
right_join.show()


+------+----+-----------+---+
|  name|  id|       name| id|
+------+----+-----------+---+
|  null|null|   Rutabaga|  1|
| Ninja|   3|      Ninja|  3|
|Pirate|   1|     Pirate|  2|
|  null|null|Darth Vader|  4|
+------+----+-----------+---+



## FULL OUTER JOIN

In [ ]:
full_outer_join = ta.join(tb, ta.name == tb.name,how='full') # Could also use 'full_outer'
full_outer_join.show()

+---------+----+-----------+----+
|     name|  id|       name|  id|
+---------+----+-----------+----+
|     null|null|   Rutabaga|   1|
|Spaghetti|   4|       null|null|
|    Ninja|   3|      Ninja|   3|
|   Pirate|   1|     Pirate|   2|
|   Monkey|   2|       null|null|
|     null|null|Darth Vader|   4|
+---------+----+-----------+----+



In [ ]:
from pyspark.sql.functions import col
full_outer_join.filter(col('tb.name').isNull() | col('ta.name').isNull()).show()


+---------+----+-----------+----+
|     name|  id|       name|  id|
+---------+----+-----------+----+
|     null|null|   Rutabaga|   1|
|Spaghetti|   4|       null|null|
|   Monkey|   2|       null|null|
|     null|null|Darth Vader|   4|
+---------+----+-----------+----+

